In [ ]:
import numpy as np
import tensorflow as tf

# Define the environment
class Environment:
    def __init__(self):
        self.state = 0
        self.end_states = [3, 6]

    def step(self, action):
        if self.state in self.end_states:
            return self.state, 0, True
        self.state += action
        return self.state, -1, self.state in self.end_states

# Define the Q-Learning algorithm
class QLearning:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.q_table = tf.Variable(tf.random.uniform(shape=(state_size, action_size), minval=0, maxval=1))

    def get_action(self, state, epsilon):
        if np.random.random() < epsilon:
            return np.random.randint(self.action_size)
        else:
            return tf.argmax(self.q_table[state]).numpy()

    def update(self, state, action, reward, next_state, alpha, gamma):
        q_next = tf.reduce_max(self.q_table[next_state])
        q_val = self.q_table[state][action]
        q_update = q_val + alpha * (reward + gamma * q_next - q_val)
        self.q_table = tf.tensor_scatter_nd_update(self.q_table, [[state, action]], [q_update])

# Define the training loop
def train(agent, env, episodes, alpha, gamma, epsilon):
    for episode in range(episodes):
        state = 0
        total_reward = 0
        while True:
            action = agent.get_action(state, epsilon)
            agent.update(state, action, reward, next_state, alpha, gamma)
            next_state, reward, done = env.step(action)
            state = next_state
            total_reward += reward
            if done:
                break
        print(f"Episode {episode + 1}: Total Reward = {total_reward}")

# Define the main function
def main():
    env = Environment()
    agent = QLearning(state_size=7, action_size=3)
    train(agent, env, episodes=1000, alpha=0.1, gamma=0.9, epsilon=0.1)

if __name__ == "__main__":
    main()
